## 第五章练习题

In [303]:
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sum, Alias, Domain, Sense
from gamspy.math import log
import pandas as pd
pd.options.display.float_format = '{:.1f}'.format 

### container

In [304]:
m = Container()

### sets

In [305]:
ac = Set(m,name="ac",description="all units", records=["sec1","sec2","sec3","lab","cap","hh","total","tartot"])
acc = Alias(m,name="acc", alias_with=ac)
i = Set(m,name="i", description="units", domain=ac, records=["sec1","sec2","sec3","lab","cap","hh"])
j = Alias(m, name="j", alias_with=i)
ac.records, i.records

(      uni element_text
 0    sec1             
 1    sec2             
 2    sec3             
 3     lab             
 4     cap             
 5      hh             
 6   total             
 7  tartot             ,
      ac element_text
 0  sec1             
 1  sec2             
 2  sec3             
 3   lab             
 4   cap             
 5    hh             )

### data

In [306]:
data = pd.read_excel("ch5.xlsx", index_col=0) 
data0 = data.stack().reset_index() 
data0


,level_0,level_1,0
0,sec1,sec1,160.0
1,sec1,sec2,150.0
2,sec1,sec3,90.0
3,sec1,hh,540.0
4,sec1,total,940.0
5,sec1,tartot,920.0
6,sec2,sec1,140.0
7,sec2,sec2,320.0
8,sec2,sec3,170.0
9,sec2,hh,910.0


In [307]:
data1 = data.iloc[0:-2,0:-2].stack().reset_index()
data1 

,level_0,level_1,0
0,sec1,sec1,160.0
1,sec1,sec2,150.0
2,sec1,sec3,90.0
3,sec1,hh,540.0
4,sec2,sec1,140.0
5,sec2,sec2,320.0
6,sec2,sec3,170.0
7,sec2,hh,910.0
8,sec3,sec1,80.0
9,sec3,sec2,150.0


In [308]:
data2 = data["tartot"].iloc[:-2].reset_index()
data2 

,index,tartot
0,sec1,920.0
1,sec2,1540.0
2,sec3,2000.0
3,lab,2000.0
4,cap,900.0
5,hh,2900.0


### Parameters

In [309]:
sam = Parameter(m, name="sam", description="intial value", domain=[ac,acc], records=data0)
Q0 = Parameter(m, name="Q0", description="intial value", domain=[i,j], records=data1)
tartot = Parameter(m, name="tartot",description="target total",domain=i, records=data2)
condis = Parameter(m, name="condis", description="column difference", domain=j)
rowdis = Parameter(m, name="rowdis", description="row difference", domain=i)
H0 = Parameter(m, name="H0", description="sum inital value")
H0[...] = Sum([i,j],Q0[i,j])

### Variables

In [310]:
Q = Variable(m, name="Q",description="adjusted value",domain=[i,j],type="positive")
H = Variable(m, name="H",description="adjusted SUM value", type="positive")
Hratio = Variable(m, name="Hratio",description="adjusted SUM value to initial", type="positive")

### Equations

In [311]:
balance = Equation(m, name="balance", description="balance equation", domain=i)
balance[i] = Sum(j.where[Q0[i,j]>0], Q[i,j])  == Sum(j.where[Q0[j,i]>0], Q[j,i])

Heq = Equation(m, name="Heq", description="H equation")
Heq[...] = H == Sum(Domain(i,j).where[Q0[i,j]>0],Q[i,j])

Hratioeq = Equation(m, name="Hratioeq", description="H ratio equation")
Hratioeq[...] = Hratio == H/H0

### Objective

In [312]:
obj = Sum(Domain(i,j).where[Q0[i,j]>0],(Q[i,j]-Q0[i,j])**2)
obj1 = Sum(Domain(i,j).where[Q0[i,j]>0],(1/H)*Q[i,j]*log(Q[i,j]/Q0[i,j])) - log(Hratio)

### Model

In [313]:
# 第一题
sambal = Model(m, name="sambal", equations=[balance], problem="NLP", sense=Sense.MIN, objective=obj)
Q.l[i,j] = Q0[i,j]
sambal.solve()
result1 = Q.records
z1 = sambal.objective_value

In [314]:
# 第二题
sambal1 = Model(m, name="sambal1", equations=[balance], problem="NLP", sense=Sense.MIN, objective=obj)
Q.l[i,j] = Q0[i,j]
Q.fx[i,"hh"] = Q0[i,"hh"]
sambal1.solve()
result2 = Q.records
z2 = sambal1.objective_value

In [315]:
# 第四题
iter = 1 
maxdis = 0.1
while iter<5000 and maxdis>1e-12:
    sam["total",j] = Sum(i,sam[i,j])
    sam[i,j] = sam[i,j]/sam["total",j]*sam["tartot",j]

    sam[i,"total"] = Sum(j,sam[i,j])
    sam[i,j] = sam[i,j]/sam[i,"total"]*sam[i,"tartot"]

    condis[j] = abs(Sum(i,sam[i,j]) - sam["total",j])
    rowdis[i] = abs(Sum(j,sam[i,j]) - sam[i,"total"])

    c = condis.records.iloc[:,1].values.max()
    r = rowdis.records.iloc[:,1].values.max()

    maxdis = max(c,r)



In [316]:
# 第五题
sambal2 = Model(m, name="sambal2", equations=[balance,Heq,Hratioeq], problem="NLP", sense=Sense.MIN, objective=obj1)
Q.l[i,j] = Q0[i,j]
H.l[...] = H0
Hratio.lo[...] = 0.5
Hratio.up[...] = 2

sambal2.solve()
result3 = Q.records
z3 = sambal2.objective_value

j,sec1,sec2,sec3,lab,cap,hh
i,,,,,,
sec1,167.1,148.0,90.5,NaN,NaN,540.0
sec2,154.9,334.3,181.0,NaN,NaN,910.0
sec3,86.8,153.7,261.2,NaN,NaN,610.0
lab,348.9,370.6,419.7,NaN,NaN,0.0
cap,187.8,573.6,159.4,NaN,NaN,0.0
hh,NaN,NaN,NaN,1139.1,920.9,0.0


In [320]:
### results

In [322]:
# 第一题
result1.iloc[:,0:3].pivot(index="i",columns="j",values="level")

j,sec1,sec2,sec3,lab,cap,hh
i,,,,,,
sec1,160.0,143.3,85.6,NaN,NaN,505.2
sec2,146.7,320.0,172.2,NaN,NaN,881.9
sec3,84.4,147.8,250.0,NaN,NaN,579.6
lab,324.0,357.3,399.5,NaN,NaN,NaN
cap,179.0,552.3,154.5,NaN,NaN,NaN
hh,NaN,NaN,NaN,1080.8,885.8,NaN


In [325]:
z1

4812.962962962966

In [323]:
# 第二题
result2.iloc[:,0:3].pivot(index="i",columns="j",values="level")

j,sec1,sec2,sec3,lab,cap,hh
i,,,,,,
sec1,160.0,142.5,85.0,NaN,NaN,540.0
sec2,147.5,320.0,172.5,NaN,NaN,910.0
sec3,85.0,147.5,250.0,NaN,NaN,610.0
lab,340.0,372.5,415.0,NaN,NaN,0.0
cap,195.0,567.5,170.0,NaN,NaN,0.0
hh,NaN,NaN,NaN,1127.5,932.5,0.0


In [324]:
z2

13550.0

In [328]:
# 第四题
sam.records.pivot(index="ac",columns="acc",values="value")

acc,sec1,sec2,sec3,lab,cap,hh,total,tartot
ac,,,,,,,,
sec1,105.1,99.6,99.2,NaN,NaN,616.1,920.0,920.0
sec2,92.6,213.9,188.5,NaN,NaN,1045.0,1540.0,1540.0
sec3,93.5,177.3,490.3,NaN,NaN,1238.9,2000.0,2000.0
lab,472.3,537.3,990.4,NaN,NaN,NaN,2000.0,2000.0
cap,156.5,511.9,231.6,NaN,NaN,NaN,900.0,900.0
hh,NaN,NaN,NaN,2000.0,900.0,NaN,2900.0,2900.0
total,920.0,1540.0,2000.0,2000.0,900.0,2900.0,NaN,NaN
tartot,920.0,1540.0,2000.0,2000.0,900.0,2900.0,NaN,NaN


In [329]:
# 第五题
result3.iloc[:,0:3].pivot(index="i",columns="j",values="level")

j,sec1,sec2,sec3,lab,cap,hh
i,,,,,,
sec1,167.1,148.0,90.5,NaN,NaN,540.0
sec2,154.9,334.3,181.0,NaN,NaN,910.0
sec3,86.8,153.7,261.2,NaN,NaN,610.0
lab,348.9,370.6,419.7,NaN,NaN,0.0
cap,187.8,573.6,159.4,NaN,NaN,0.0
hh,NaN,NaN,NaN,1139.1,920.9,0.0


In [330]:
z3

0.0005719841638003806